# Squirrelwaffle Config Extraction

Notes from our live stream where we reverse engineered the squirrelwaffle loader and built a static config extractor for it.

<a href="https://www.youtube.com/watch?v=9X2P7aFKSw0" target="_blank"><img src="http://img.youtube.com/vi/9X2P7aFKSw0/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>

## References
- [Malware Traffic Analysis Sample 1](https://www.malware-traffic-analysis.net/2021/09/17/index.html)
- [Malware Traffic Analysis Sample 2](https://www.malware-traffic-analysis.net/2021/09/21/index2.html)
- [Unpacked Sample (malshare)](https://malshare.com/sample.php?action=detail&hash=6095f96dd5eca96a3fb9338eec4ab574921c0febb36f6a6db60aae1aeb9ffcab)

## Setup

Make sure you have pefile installed `pip install pefile` and you change the `SAMPLE_PATH` to the squirrelwaffle sample you have downloaded. 


In [4]:
import pefile

In [2]:
SAMPLE_PATH = r'/tmp/squirrel.bin'
data = open(SAMPLE_PATH,'rb').read()

## Extract the .rdata section

The `.rdata` section has the encrypted data and the decryption keys so let's grab that first.

In [5]:
pe = pefile.PE(data=data)
rdata = None
for s in pe.sections:
    if b'.rdata' in s.Name:
        rdata = s.get_data()
len(rdata)

20480

## Parse .rdata

The encrypted strings in the `.rdata` section are followed by their key and all data, keys, and strings are seperated by null bytes. We can parse out the data and key by splitting the `.rdata` on null bytes then looking for the largest blocks of data. These are likely the encrypted blocklist and the encrypted c2 list. 

We want to maintain two lists of data blocks, one sorted and one original order. This way we can use the sorted list to get the largest blocks, and the original order list to find the key which is sequentially after the encrypted data.

In [6]:
blocks = rdata.split(b'\x00')
blocks = [x for x in blocks if x != b'']
blocks_sorted = sorted(blocks, key=len)

## Decryption function

The config decryption is a simple xor so we want to replicate this in python


In [7]:
def decrypt(key, data):
    out = ''
    for i in range(len(data)):
        out += chr(data[i] ^ key[i % len(key)])
    return out

## Decrypt the blocklist

The largest encrupted block is likely the blocklist so let's decrypt that first.

In [8]:
for i in range(len(blocks)):
    if blocks[i] == blocks_sorted[-1]:
        out = decrypt(blocks[i+1], blocks[i])
print(out)

94.46.179.80
206.189.205.251
88.242.66.45
85.75.110.214
87.104.3.136
207.244.91.171
49.230.88.160
91.149.252.75
91.149.252.88
92.211.109.152
178.0.250.168
88.69.16.230
95.223.77.160
99.234.62.23
2.206.105.223
84.222.8.201
89.183.239.142
5.146.132.101
77.7.60.154
45.41.106.122
45.74.72.13
74.58.152.123
88.87.68.197
109.70.100.25
185.67.82.114
207.102.138.19
204.101.161.14
193.128.108.251
111.7.100.17
111.7.100.16
74.125.210.62
74.125.210.36
104.244.74.57
185.220.101.145
185.220.101.144
185.220.101.18
185.220.100.246
185.220.101.228
185.220.100.243
185.220.101.229
185.220.101.147
185.220.102.250
185.220.100.241
199.195.251.84
213.164.204.94
74.125.213.7
74.125.213.9
185.220.100.249
37.71.173.58
93.2.220.100
188.10.191.109
81.36.17.247
70.28.47.118
45.133.172.222
108.41.227.196
37.235.53.46
162.216.47.22
154.3.42.51
45.86.200.60
212.230.181.152
185.192.70.11
37.142.65.69
87.166.51.31
178.198.76.175
128.90.172.136
172.58.227.224
201.77.112.133
64.124.12.162
87.166.51.28
104.244.72.115
109.

## Decrypt the c2s

The second largest encrupted block is likely the C2 list so let's decrypt that now.

In [9]:
for i in range(len(blocks)):
    if blocks[i] == blocks_sorted[-2]:
        out = decrypt(blocks[i+1], blocks[i])
print(out)

celulasmadreenmexico.com.mx/Wt793Aua|gerencial.institutoacqua.org.br/XynFkhJAxnm|dashboard.adlytic.ai/LlvLoc9O3|bussiness-z.ml/3pdEiqsni|ifiengineers.com/hGVc55g2e|bonusvulkanvegas.srdm.in/U7oOxmI1m|ebrouteindia.com/JEqGe1hNR|test.dirigu.ro/dXf4cS4GPL|cablingpoint.com/LjDG0hkp|perfectdemos.com/T6PQGYCMt|afrizam.360cyberlink.com/f36rjSN5D1|giasuphire.tddvn.com/miFO43YP9b|priyacareers.com/GiTHMPbU|assurant.360cyberlink.com/DGx4k8U9Hil|sig.institutoacqua.org.br/tM7tINg2sCU|


## Clean up the c2 list

It looks like the C2 list is seperated by pipes `|` so let's turn that into a nice list.

In [13]:
print(out.replace('|','\n'))

celulasmadreenmexico.com.mx/Wt793Aua
gerencial.institutoacqua.org.br/XynFkhJAxnm
dashboard.adlytic.ai/LlvLoc9O3
bussiness-z.ml/3pdEiqsni
ifiengineers.com/hGVc55g2e
bonusvulkanvegas.srdm.in/U7oOxmI1m
ebrouteindia.com/JEqGe1hNR
test.dirigu.ro/dXf4cS4GPL
cablingpoint.com/LjDG0hkp
perfectdemos.com/T6PQGYCMt
afrizam.360cyberlink.com/f36rjSN5D1
giasuphire.tddvn.com/miFO43YP9b
priyacareers.com/GiTHMPbU
assurant.360cyberlink.com/DGx4k8U9Hil
sig.institutoacqua.org.br/tM7tINg2sCU

